In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
import warnings
import gc
import time
import sys
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

from sklearn.utils import shuffle



In [3]:
train= pd.read_csv("../input/train.csv")
test= pd.read_csv("../input/test.csv")
test_lgbm = pd.read_csv("../input/output_test_lgbm_base1.csv") # contains the lgbm prediction on the test set
lgbm = pd.read_csv("../input/output_lgbm_base1.csv")   #contains the lgbm predictions on the train set

#squash together to give both sets an additional column from the lgbm

train_df = train.copy()
test_df = test.copy()
train_df["lgbm_out"] = lgbm["output"]
test_df["lgbm_out"] = test_lgbm["target"]

train_df.info()
test_df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 203 entries, ID_code to lgbm_out
dtypes: float64(201), int64(1), object(1)
memory usage: 309.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 202 entries, ID_code to lgbm_out
dtypes: float64(201), object(1)
memory usage: 308.2+ MB


In [19]:
feats = ["var_{}".format(i) for i in range(200)]
feats.append("lgbm_out")
targets2 = ["target{}".format(i) for i in range(0,2)]

split_tar_df = train_df.copy(deep=True)
split_tar_df['target1'] = train_df['target']
split_tar_df['target0'] = 1 - train_df['target']
split_tar_df.drop(columns=['target'])

zero_subset_df = split_tar_df[split_tar_df['target0'] > 0.9]
ones_subset_df = split_tar_df[split_tar_df['target1'] > 0.9]

#Balance the data by adding more ones.
frames = [zero_subset_df]
for i in range(0,4):
    frames.append(ones_subset_df)  #NN will add Gaussian noise.

combined_df = pd.concat(frames)
combined_df = shuffle(combined_df)

combined_df.info()

X = combined_df[feats]
X_test = test_df[feats]
y = combined_df[targets2]


scaler = StandardScaler()

X_sc = scaler.fit_transform(X)
X_test_sc = scaler.fit_transform(X_test)

print(X_sc.shape)
print(y.shape)



<class 'pandas.core.frame.DataFrame'>
Int64Index: 260294 entries, 129009 to 196381
Columns: 205 entries, ID_code to target0
dtypes: float64(201), int64(3), object(1)
memory usage: 409.1+ MB
(260294, 201)
(260294, 2)


In [20]:
import keras
from keras.layers import (Flatten, Conv1D, Conv2D, Input, Dense, Dropout, BatchNormalization,
                          concatenate, GaussianNoise, Reshape, TimeDistributed, LeakyReLU, PReLU, Embedding)
from keras.models import Model, load_model, save_model
from keras.optimizers import SGD, Adam
from sklearn.base import BaseEstimator, ClassifierMixin
from pathlib import Path
from keras.callbacks import Callback

class ROC_AUC(Callback):
    def __init__(self, validation_data):
        self.X_val, self.y_val = validation_data
    
    def on_epoch_end(self, epoch, logs={}):
        print("ROC AUC for this fold is ", roc_auc_score(self.y_val, self.model.predict(X_val)))
        
class NNv1(BaseEstimator, ClassifierMixin):
    def __init__(self,
                 inp_shape=len(feats),
                 gaussian_noise=0.02,
                 dense1_dim=128,
                 dense2_dim=128,
                 dense3_dim=64,
                 dense1_kwargs=None,
                 dense2_kwargs=None,
                 dense3_kwargs=None,
                 classifier_kwargs=None,
                 optimizer=SGD,
                 opt_kwargs=None,
                 ):
        self.inp_shape = inp_shape
        self.gaussian_noise = gaussian_noise
        self.dense1_dim = dense1_dim
        self.dense2_dim = dense2_dim
        self.dense3_dim = dense3_dim
        self.dense1_kwargs = dense1_kwargs
        self.dense2_kwargs = dense2_kwargs
        self.dense3_kwargs = dense3_kwargs
        self.classifier_kwargs = classifier_kwargs
        self.optimizer = optimizer
        self.opt_kwargs = opt_kwargs
        self._default_initiaization()

    def _default_initiaization(self):
        if self.dense1_kwargs is None:
            self.dense1_kwargs = {"kernel_initializer": "glorot_uniform"}
        if self.dense2_kwargs is None:
            self.dense2_kwargs = {"kernel_initializer": "he_uniform"}
        if self.classifier_kwargs is None:
            self.classifier_kwargs = {"kernel_initializer": "he_uniform"}
        if self.opt_kwargs is None:
            self.opt_kwargs = {}

    def _build_model(self):
        inp = Input(shape=(self.inp_shape,))
        x = GaussianNoise(self.gaussian_noise)(inp)
        x = Reshape((self.inp_shape, 1))(inp)
        x = Dense(self.dense1_dim, activation='relu',)(x)
        #d1 = TimeDistributed(Dropout(0.2))(d1)
        x = Dense(self.dense1_dim, activation='relu',)(x)
        #d2 = PReLU()(d2)
        #d2 = TimeDistributed(Dropout(0.2))(d2)
        #x = concatenate([d1, d2])
        x = Flatten()(x)
        out = Dense(2, activation='softmax', **self.classifier_kwargs)(x)

        model = Model(inputs=inp, outputs=out)
        opt = self.optimizer(**self.opt_kwargs)
        model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
        return model

    def fit(self, X, y, *args, **kwargs):
        self.model = self._build_model()
        print(self.model.summary())
        self.model.fit(X, y, *args, **kwargs)
        return self

    def predict(self, X, y=None, weight_path=None, **kwargs):
        if self.model:
            if weight_path is not None:
                self.model.load_weights(weight_path)
            y_hat = self.model.predict(X, **kwargs)
        else:
            raise ValueError("Model not fit yet")
        return y_hat

In [21]:
y_test_pred = np.zeros(len(test))
    
X_train, X_valid, y_train, y_valid = train_test_split(X_sc, y, test_size=0.2, random_state=42)

model = NNv1(opt_kwargs = {"lr": 0.02, "momentum": 0.9, "nesterov": True, "clipnorm": 1})

model.fit(X_train, y_train, epochs=300, batch_size = 1000, validation_data =(X_valid, y_valid))
pred = model.predict(X_valid)
print(pred.shape)
print( "  auc = ", roc_auc_score(y_valid, pred) )
y_test_pred = model.predict(X_test_sc)[:,1]
    

#save base submission
sub_df1 = pd.DataFrame({"ID_code":test["ID_code"].values})
sub_df1["target"] = y_test_pred 
sub_df1.to_csv("submission_full_nn_lgbm_guide.csv", index=False)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 201)               0         
_________________________________________________________________
reshape_7 (Reshape)          (None, 201, 1)            0         
_________________________________________________________________
dense_19 (Dense)             (None, 201, 128)          256       
_________________________________________________________________
dense_20 (Dense)             (None, 201, 128)          16512     
_________________________________________________________________
flatten_7 (Flatten)          (None, 25728)             0         
_________________________________________________________________
dense_21 (Dense)             (None, 2)                 51458     
Total params: 68,226
Trainable params: 68,226
Non-trainable params: 0
_________________________________________________________________
None
T

Epoch 51/300
208235/208235 [==============================] - 6s 27us/step - loss: 0.1687 - acc: 0.9351 - val_loss: 0.1653 - val_acc: 0.9394
Epoch 52/300
208235/208235 [==============================] - 6s 27us/step - loss: 0.1683 - acc: 0.9353 - val_loss: 0.1676 - val_acc: 0.9335
Epoch 53/300
208235/208235 [==============================] - 6s 27us/step - loss: 0.1681 - acc: 0.9350 - val_loss: 0.1667 - val_acc: 0.9386
Epoch 54/300
208235/208235 [==============================] - 6s 27us/step - loss: 0.1671 - acc: 0.9356 - val_loss: 0.1649 - val_acc: 0.9350
Epoch 55/300
208235/208235 [==============================] - 6s 27us/step - loss: 0.1666 - acc: 0.9361 - val_loss: 0.1611 - val_acc: 0.9397
Epoch 56/300
208235/208235 [==============================] - 6s 27us/step - loss: 0.1653 - acc: 0.9361 - val_loss: 0.1569 - val_acc: 0.9390
Epoch 57/300
208235/208235 [==============================] - 6s 27us/step - loss: 0.1649 - acc: 0.9367 - val_loss: 0.1691 - val_acc: 0.9385
Epoch 58/300


208235/208235 [==============================] - 6s 27us/step - loss: 0.1335 - acc: 0.9496 - val_loss: 0.1282 - val_acc: 0.9550
Epoch 110/300
208235/208235 [==============================] - 6s 27us/step - loss: 0.1319 - acc: 0.9503 - val_loss: 0.1340 - val_acc: 0.9483
Epoch 111/300
208235/208235 [==============================] - 6s 27us/step - loss: 0.1319 - acc: 0.9504 - val_loss: 0.1325 - val_acc: 0.9532
Epoch 112/300
208235/208235 [==============================] - 6s 27us/step - loss: 0.1312 - acc: 0.9505 - val_loss: 0.1360 - val_acc: 0.9475
Epoch 113/300
208235/208235 [==============================] - 6s 27us/step - loss: 0.1305 - acc: 0.9511 - val_loss: 0.1313 - val_acc: 0.9537
Epoch 114/300
208235/208235 [==============================] - 6s 27us/step - loss: 0.1299 - acc: 0.9510 - val_loss: 0.1357 - val_acc: 0.9466
Epoch 115/300
208235/208235 [==============================] - 6s 27us/step - loss: 0.1291 - acc: 0.9515 - val_loss: 0.1278 - val_acc: 0.9545
Epoch 116/300
208235

Epoch 167/300
208235/208235 [==============================] - 6s 27us/step - loss: 0.1073 - acc: 0.9607 - val_loss: 0.1099 - val_acc: 0.9636
Epoch 168/300
208235/208235 [==============================] - 6s 27us/step - loss: 0.1068 - acc: 0.9609 - val_loss: 0.1100 - val_acc: 0.9569
Epoch 169/300
208235/208235 [==============================] - 6s 27us/step - loss: 0.1064 - acc: 0.9612 - val_loss: 0.1034 - val_acc: 0.9640
Epoch 170/300
208235/208235 [==============================] - 6s 27us/step - loss: 0.1060 - acc: 0.9610 - val_loss: 0.1113 - val_acc: 0.9566
Epoch 171/300
208235/208235 [==============================] - 6s 27us/step - loss: 0.1056 - acc: 0.9612 - val_loss: 0.1050 - val_acc: 0.9648
Epoch 172/300
208235/208235 [==============================] - 6s 27us/step - loss: 0.1052 - acc: 0.9616 - val_loss: 0.1133 - val_acc: 0.9558
Epoch 173/300
208235/208235 [==============================] - 6s 27us/step - loss: 0.1050 - acc: 0.9620 - val_loss: 0.1011 - val_acc: 0.9651
Epoch 

208235/208235 [==============================] - 6s 27us/step - loss: 0.0902 - acc: 0.9678 - val_loss: 0.0860 - val_acc: 0.9714
Epoch 225/300
208235/208235 [==============================] - 6s 27us/step - loss: 0.0896 - acc: 0.9677 - val_loss: 0.0965 - val_acc: 0.9636
Epoch 226/300
208235/208235 [==============================] - 6s 27us/step - loss: 0.0898 - acc: 0.9677 - val_loss: 0.0854 - val_acc: 0.9715
Epoch 227/300
208235/208235 [==============================] - 6s 27us/step - loss: 0.0891 - acc: 0.9684 - val_loss: 0.0948 - val_acc: 0.9637
Epoch 228/300
208235/208235 [==============================] - 6s 27us/step - loss: 0.0891 - acc: 0.9681 - val_loss: 0.0890 - val_acc: 0.9706
Epoch 229/300
208235/208235 [==============================] - 6s 27us/step - loss: 0.0885 - acc: 0.9684 - val_loss: 0.0918 - val_acc: 0.9649
Epoch 230/300
208235/208235 [==============================] - 6s 27us/step - loss: 0.0890 - acc: 0.9678 - val_loss: 0.0830 - val_acc: 0.9731
Epoch 231/300
208235

208235/208235 [==============================] - 6s 27us/step - loss: 0.0775 - acc: 0.9726 - val_loss: 0.0864 - val_acc: 0.9658
Epoch 282/300
208235/208235 [==============================] - 6s 27us/step - loss: 0.0768 - acc: 0.9727 - val_loss: 0.0805 - val_acc: 0.9721
Epoch 283/300
208235/208235 [==============================] - 6s 27us/step - loss: 0.0769 - acc: 0.9725 - val_loss: 0.0815 - val_acc: 0.9684
Epoch 284/300
208235/208235 [==============================] - 6s 27us/step - loss: 0.0777 - acc: 0.9721 - val_loss: 0.0725 - val_acc: 0.9766
Epoch 285/300
208235/208235 [==============================] - 6s 27us/step - loss: 0.0759 - acc: 0.9733 - val_loss: 0.0813 - val_acc: 0.9689
Epoch 286/300
208235/208235 [==============================] - 6s 27us/step - loss: 0.0765 - acc: 0.9729 - val_loss: 0.0754 - val_acc: 0.9756
Epoch 287/300
208235/208235 [==============================] - 6s 27us/step - loss: 0.0757 - acc: 0.9729 - val_loss: 0.0811 - val_acc: 0.9696
Epoch 288/300
208235